## Radiotoxicity chart tool
### Isotope-based Spent Fuel Simulation

### 1. Initialization

In [ ]:
# Imports
import sys
import importlib
import numpy as np
from pathlib import Path

#Project directories
sys.path.append(str(Path().resolve() / "Backend" / "Isotope-based Spent Fuel Simulation"))
import find_project_root
importlib.reload(find_project_root)
from find_project_root import find_project_root
project_root = find_project_root()

backend_path = project_root / "Backend"
main_output_dir = project_root / "Simulation results"

In [ ]:
#Resource file Paths
excel_file_fuel = project_root / "Resources" / "SpentFuel_Data.xlsx"
excel_file_DC = project_root / "Resources" / "Dose coefficients.xlsx"

chain_file = project_root / "Resources" / "chain_endfb71_pwr.xml"

In [ ]:
#Reactor selection
selected_sheet = "B.13 - MOX"

#DC selection
selected_DC = "Adults" #Adults or Infants

### 2. Simulation of Spent Fuel Isotopes Decay (only needed if concentration data not already existing)

In [ ]:
# Time steps
time_steps = np.concatenate((
    np.logspace(0, 3, num=80, endpoint=False),  # 1 to nearly 1000
    np.logspace(3, 6, num=40)                  # from 1000 to 1e6
))

In [ ]:
#Run Simulation
from simulation import simulation
simulation(project_root, main_output_dir, excel_file_fuel, time_steps, selected_sheet, selected_DC, chain_file)

### 3. Select Data Path (Plot Input Data/ RT Calculation Input Data and Output Path)

In [ ]:
#Show file chooser
from file_chooser import file_chooser
chooser = file_chooser(main_output_dir)

In [ ]:
#Show selected path
print("Selected path:", chooser.selected_path)

### 4. Radiotoxicity calculation (only needed if RT data not already existing)

In [ ]:
#Run RT calculation
from RT_calculation import RT_calculation
rt_data = RT_calculation(chooser, excel_file_fuel, selected_sheet, excel_file_DC, selected_DC)

### 5. Plotting

### 5.1 Plot configuration

In [ ]:
save_plot = False

In [ ]:
#Cooling time/Data offset
offset = 0

In [ ]:
#Graph settings
plot_config = {

    #Graph linestyle
    "linewidth" : 2.0,

    #Figure size
    "fig_width" : 12,
    "fig_height" : 7,

    #Axis
    "plot_axis_range_x" : [1e0, 1e6],
    "plot_axis_range_y" : [1e0, 1e11],

    "xticks_fontsize" : 12,
    "yticks_fontsize" : 12,

    #Titles
    "plot_title" : f"Radiotoxicity of Spent Fuel incl. Decay Products", #no slashes allowed

    "plot_subtitle" : "SF at discharge, MOX, 65 GWd/t, Adults",
    #"plot_subtitle" : "SF at discharge, UO2, 4.5%, 65 GWd/t, Adults",

    "plot_xlabel" : "Time (years)",
    "plot_ylabel" : "Radiotoxicity [Sv/t$_{IHM}$]",

    #Footnote
    "show_footnote" : True,
    "footnote_text" : "IHM = Initial Heavy Metal\nSF = Spent Fuel\nPart. eff.s: U = 99.9%, Pu = 99.5%, MA = 99.95%",

    #Font name and sizes
    "font_name" : 'DejaVu Sans',
    "title_fontsize" : 14,
    "label_fontsize" : 13,
    "legend_fontsize" : 12,
    "subtitle_fontsize" : 12,

    #Legend location
    "show_legend" : True,
    "plot_legend_loc" : 'upper right',
    "legend_outside" : False,  #False = inside

    #Grid
    "plot_grid_show" : True,
    "plot_grid_which" : 'major',
    "plot_grid_linestyle" : '--',
    "plot_grid_linewidth" : 0.5,

    #resolution
    "dpi" : 300,
}

In [ ]:
# Show UI & checkbox for additional FP curve
from plot_UI import plot_UI
checkboxes, color_pickers = plot_UI(chooser)

from show_fp2 import show_fp2
show_fp2()

In [ ]:
# Reference value configuration
ref_values = {
    "ref_value1": 256000,  #Ref value for 8.41t of natural U in equlibrium with its daughter products (ingestion/adults)
    #"ref_value2": ...
}
    
ref_config = {
    "ref_value1": {"label": "Natural Uranium (8.41t)", "linestyle": "--", "linewidth": 2, "color": "#ee0000"},
    #"ref_value2": ...
    "legend_bbox_anchor": (1.287, 1),  #horizontal position of legend if outside
}

### 5.2 Plot chart

In [ ]:
# Plot
from plot import plot
plot(chooser, checkboxes, color_pickers, excel_file_fuel, selected_sheet, selected_DC, save_plot, offset, plot_config, ref_values, ref_config,
             
             partition_eff_u=1, partition_eff_pu=0.995, partition_eff_ma=1, #set partiotionig efficiencies
             
             show_fp2=show_fp2_checkbox.value, second_fp_path=fp2_chooser.selected_path, fp2_colorpicker=fp2_colorpicker, fp2_label=fp2_label)

#### Read concentrations of parent nuclide & decay products

In [ ]:
target_nuclide_1 = "H3" #change to desired nuclide

In [ ]:
from read_conc_parent_DPs import read_conc_parent_DPs
read_conc_parent_DPs(chooser, target_nuclide_1)

#### Read nuclide concentration across all parent nuclides

In [ ]:
target_nuclide_2 = "I129" #change to desired nuclide

In [ ]:
from read_nuc_conc_across import read_nuc_conc_across
read_nuc_conc_across(chooser, target_nuclide_2)

#### Read RTs

In [ ]:
from read_RTs import read_RTs
read_RTs(chooser)